In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName('restaurant-average')
sc = SparkContext(conf = conf)

In [3]:
directory = '/Users/admin/Desktop/spark_hadoop/data-engineering/01-spark'
filename = 'restaurant_reviews.csv'

lines = sc.textFile(f"file:///{directory}/{filename}")


lines.collect()

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [7]:
header = lines.first()

filtered_lines = lines.filter(lambda x:x != header)

filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [9]:
def parse(row):
    col = row.split(',')
    category = col[2]
    reviews = int(col[3])
    return (category, reviews)

category_review = filtered_lines.map(parse)

category_review.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [10]:
category_reviews_count = category_review.mapValues(lambda x: (x, 1))

category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [20]:
reduced = category_reviews_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [22]:
average = reduced.mapValues(lambda x: round(x[0]/x[1], 2))
average.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.67), ('아시안', 312.0), ('패스트푸드', 17.5)]